![Los Angeles skyline](la_skyline.jpg)

Los Angeles, California 😎. The City of Angels. Tinseltown. The Entertainment Capital of the World! 

Known for its warm weather, palm trees, sprawling coastline, and Hollywood, along with producing some of the most iconic films and songs. However, as with any highly populated city, it isn't always glamorous and there can be a large volume of crime. That's where you can help!

You have been asked to support the Los Angeles Police Department (LAPD) by analyzing crime data to identify patterns in criminal behavior. They plan to use your insights to allocate resources effectively to tackle various crimes in different areas.

## The Data

They have provided you with a single dataset to use. A summary and preview are provided below.

It is a modified version of the original data, which is publicly available from Los Angeles Open Data.

# crimes.csv

| Column     | Description              |
|------------|--------------------------|
| `'DR_NO'` | Division of Records Number: Official file number made up of a 2-digit year, area ID, and 5 digits. |
| `'Date Rptd'` | Date reported - MM/DD/YYYY. |
| `'DATE OCC'` | Date of occurrence - MM/DD/YYYY. |
| `'TIME OCC'` | In 24-hour military time. |
| `'AREA NAME'` | The 21 Geographic Areas or Patrol Divisions are also given a name designation that references a landmark or the surrounding community that it is responsible for. For example, the 77th Street Division is located at the intersection of South Broadway and 77th Street, serving neighborhoods in South Los Angeles. |
| `'Crm Cd Desc'` | Indicates the crime committed. |
| `'Vict Age'` | Victim's age in years. |
| `'Vict Sex'` | Victim's sex: `F`: Female, `M`: Male, `X`: Unknown. |
| `'Vict Descent'` | Victim's descent:<ul><li>`A` - Other Asian</li><li>`B` - Black</li><li>`C` - Chinese</li><li>`D` - Cambodian</li><li>`F` - Filipino</li><li>`G` - Guamanian</li><li>`H` - Hispanic/Latin/Mexican</li><li>`I` - American Indian/Alaskan Native</li><li>`J` - Japanese</li><li>`K` - Korean</li><li>`L` - Laotian</li><li>`O` - Other</li><li>`P` - Pacific Islander</li><li>`S` - Samoan</li><li>`U` - Hawaiian</li><li>`V` - Vietnamese</li><li>`W` - White</li><li>`X` - Unknown</li><li>`Z` - Asian Indian</li> |
| `'Weapon Desc'` | Description of the weapon used (if applicable). |
| `'Status Desc'` | Crime status. |
| `'LOCATION'` | Street address of the crime. |

In [75]:
# Re-run this cell
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
crimes = pd.read_csv("crimes.csv", parse_dates=["Date Rptd", "DATE OCC"], dtype={"TIME OCC": str})
crimes.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Weapon Desc,Status Desc,LOCATION
0,220314085,2022-07-22,2020-05-12,1110,Southwest,THEFT OF IDENTITY,27,F,B,NaN,Invest Cont,2500 S SYCAMORE AV
1,222013040,2022-08-06,2020-06-04,1620,Olympic,THEFT OF IDENTITY,60,M,H,NaN,Invest Cont,3300 SAN MARINO ST
2,220614831,2022-08-18,2020-08-17,1200,Hollywood,THEFT OF IDENTITY,28,M,H,NaN,Invest Cont,1900 TRANSIENT
3,231207725,2023-02-27,2020-01-27,0635,77th Street,THEFT OF IDENTITY,37,M,H,NaN,Invest Cont,6200 4TH AV
4,220213256,2022-07-14,2020-07-14,0900,Rampart,THEFT OF IDENTITY,79,M,B,NaN,Invest Cont,1200 W 7TH ST


In [76]:
#Initial Data Exploration

print(crimes.dtypes)
print(crimes.isna().sum())
print(crimes.describe())

DR_NO                    int64
Date Rptd       datetime64[ns]
DATE OCC        datetime64[ns]
TIME OCC                object
AREA NAME               object
Crm Cd Desc             object
Vict Age                 int64
Vict Sex                object
Vict Descent            object
Weapon Desc             object
Status Desc             object
LOCATION                object
dtype: object
DR_NO                0
Date Rptd            0
DATE OCC             0
TIME OCC             0
AREA NAME            0
Crm Cd Desc          0
Vict Age             0
Vict Sex            11
Vict Descent        10
Weapon Desc     112213
Status Desc          0
LOCATION             0
dtype: int64
              DR_NO       Vict Age
count  1.857150e+05  185715.000000
mean   2.255781e+08      39.999257
std    5.017438e+06      15.450227
min    2.009072e+08       2.000000
25%    2.210108e+08      28.000000
50%    2.220114e+08      37.000000
75%    2.310044e+08      50.000000
max    2.399097e+08      99.000000


In [77]:
# Question: Which hour has the highest frequency of crimes? Store as an integer variable called peak_crime_hour.
crimes['Hour']=crimes['TIME OCC'].str[:2].astype('int')
print(crimes.head())
peak_crime_hour=crimes['Hour'].mode().iloc[0]
peak_crime_hour

       DR_NO  Date Rptd  ...                                 LOCATION Hour
0  220314085 2022-07-22  ...  2500 S  SYCAMORE                     AV   11
1  222013040 2022-08-06  ...  3300    SAN MARINO                   ST   16
2  220614831 2022-08-18  ...                        1900    TRANSIENT   12
3  231207725 2023-02-27  ...  6200    4TH                          AV    6
4  220213256 2022-07-14  ...  1200 W  7TH                          ST    9

[5 rows x 13 columns]


12

In [78]:
# Question: Which area has the largest frequency of night crimes (crimes committed between 10pm and 3:59am)? Save as a string variable called peak_night_crime_location.

Night_Data=crimes.query('Hour >=22 | Hour <4')
peak_night_crime_location=Night_Data['AREA NAME'].mode().iloc[0]
peak_night_crime_location

'Central'

In [79]:
# Question: Identify the number of crimes committed against victims of different age groups. Save as a pandas Series called victim_ages, with age group labels "0-17", "18-25", "26-34", "35-44", "45-54", "55-64", and "65+" as the index and the frequency of crimes as the values.

Label_of_ages=["0-17", "18-25", "26-34", "35-44", "45-54", "55-64","65+"]
bins_of_ages = [0, 17, 25, 34, 44, 54, 64, float('inf')]
crimes['Age_Groups'] = pd.cut(crimes['Vict Age'], bins=bins_of_ages, labels=Label_of_ages)
print(crimes.head())
victim_ages=crimes['Age_Groups'].value_counts()
victim_ages

       DR_NO  Date Rptd  ... Hour Age_Groups
0  220314085 2022-07-22  ...   11      26-34
1  222013040 2022-08-06  ...   16      55-64
2  220614831 2022-08-18  ...   12      26-34
3  231207725 2023-02-27  ...    6      35-44
4  220213256 2022-07-14  ...    9        65+

[5 rows x 14 columns]


26-34    47470
35-44    42157
45-54    28353
18-25    28291
55-64    20169
65+      14747
0-17      4528
Name: Age_Groups, dtype: int64